In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Sales Performance Monitoring

## Description of the use case
This use case involves monitoring sales performance in real-time to track key metrics such as revenue, units sold, and average order value. By using real-time data, businesses can make informed decisions and quickly respond to market changes.

## Redis Data Structures Used
- **Hashes**: To store sales data for each product.
- **Streams**: To capture and process real-time sales transactions.
- **Sorted Sets**: To rank products based on sales performance.


In [3]:
# Mock data for the use case
sales_data = {
    'product_1001': {'name': 'Fixed Deposits', 'revenue': 5000000, 'units_sold': 100},
    'product_1002': {'name': 'Personal Loan', 'revenue': 75000000, 'units_sold': 150},
    'product_1003': {'name': 'Home Loan', 'revenue': 30000000, 'units_sold': 60},
    'product_1004': {'name': 'Savings Account', 'revenue': 2000, 'units_sold': 900}
}

# Use Hashes to store sales data for each product
for product_id, data in sales_data.items():
    r.hset(product_id, mapping=data)
    r.zadd('sales_performance', {product_id: data['revenue']})

# Use Streams to capture real-time sales transactions
r.xadd('sales_transactions', {'order_id': 'product_1001', 'revenue': 50000, 'units_sold': 2})
r.xadd('sales_transactions', {'order_id': 'product_1002', 'revenue': 5000000, 'units_sold': 3})
r.xadd('sales_transactions', {'order_id': 'product_1003', 'revenue': 7500000, 'units_sold': 2})
r.xadd('sales_transactions', {'order_id': 'product_1004', 'revenue': 25000, 'units_sold': 3})

# Function to process sales transactions and update performance metrics
def process_sales_transactions():
    stream_entries = r.xrange('sales_transactions')
    for entry in stream_entries:
        product_id = entry[1]['order_id']
        revenue = int(entry[1]['revenue'])
        units_sold = int(entry[1]['units_sold'])
        r.hincrby(product_id, 'revenue', revenue)
        r.hincrby(product_id, 'units_sold', units_sold)
        avg_order_value = int(r.hget(product_id, 'revenue')) / int(r.hget(product_id, 'units_sold'))
        r.hset(product_id, 'avg_order_value', round(avg_order_value,2))
        r.zincrby('sales_performance', revenue, product_id)

# Process the sales transactions
process_sales_transactions()

# Print the updated sales data
for product_id in sales_data.keys():
    print(f"{product_id}: {r.hgetall(product_id)}")

# Print the sales performance ranking
sales_performance = r.zrange('sales_performance', 0, -1, withscores=True, desc=True)
print('Sales Performance Ranking:')
for product in sales_performance:
    print(product)


product_1001: {'name': 'Fixed Deposits', 'revenue': '5050100', 'units_sold': '104', 'avg_order_value': '48558.65'}
product_1002: {'name': 'Personal Loan', 'revenue': '80000150', 'units_sold': '156', 'avg_order_value': '512821.47'}
product_1003: {'name': 'Home Loan', 'revenue': '37500500', 'units_sold': '64', 'avg_order_value': '585945.31'}
product_1004: {'name': 'Savings Account', 'revenue': '27004', 'units_sold': '906', 'avg_order_value': '29.81'}
Sales Performance Ranking:
('product_1002', 80000150.0)
('product_1003', 37500500.0)
('product_1001', 5050100.0)
('product_1004', 27004.0)
